In [1]:
reshape?

Object `reshape` not found.


In [3]:
from __future__ import print_function
from __future__ import division

import os
import glob
import pickle
import random
import numpy as np
import pandas as pd
import tensorflow as tf

from tqdm import tqdm

from sklearn.preprocessing import OneHotEncoder
from skimage.io import imread, imsave
from scipy.misc import imresize

flags = tf.app.flags
FLAGS = flags.FLAGS

flags.DEFINE_boolean('subset', False, 'If true, build a subset.')
flags.DEFINE_integer('downsample', 20, 'Downsample factor.')

WIDTH, HEIGHT = 2*640 // FLAGS.downsample, 2*480 // FLAGS.downsample
NUM_CLASSES = 10


In [4]:
#home_folder = os.path.expanduser("~")
data_folder = os.path.join("~/deeplearning/kaggle/distracted-drivers-tf-master/dataset/driver_imgs_list.csv")
print (data_folder)
driver_imgs_list = pd.read_csv(data_folder)
driver_imgs_grouped = driver_imgs_list.groupby('classname')
driver_imgs_grouped

~/deeplearning/kaggle/distracted-drivers-tf-master/dataset/driver_imgs_list.csv


In [17]:
paths = os.path.join('c0/') + driver_ids_group.img


In [5]:
driver_ids_group = driver_imgs_grouped.get_group('c0')

In [7]:
driver_ids_group[:100]

,subject,classname,img
0,p002,c0,img_44733.jpg
1,p002,c0,img_72999.jpg
2,p002,c0,img_25094.jpg
3,p002,c0,img_69092.jpg
4,p002,c0,img_92629.jpg
5,p002,c0,img_3370.jpg
6,p002,c0,img_67639.jpg
7,p002,c0,img_58560.jpg
8,p002,c0,img_35779.jpg
9,p002,c0,img_10012.jpg


In [28]:
driver_ids = []
driver_ids = driver_ids_group['subject'].tolist()


In [49]:
data_folder = os.path.join("~/deeplearning/kaggle/distracted-drivers-tf-master/dataset/driver_imgs_list.csv")
print (data_folder)

~/deeplearning/kaggle/distracted-drivers-tf-master/dataset/driver_imgs_list.csv


In [23]:
def load_image(path):
    return imresize(imread(path), (HEIGHT, WIDTH))

def load_train(base):
    driver_imgs_list = pd.read_csv('driver_imgs_list.csv')
    driver_imgs_grouped = driver_imgs_list.groupby('classname')

    X_train = []
    y_train = []
    driver_ids = []

    print('Reading train images...')
    for j in range(NUM_CLASSES):
        print('Loading folder c{}...'.format(j))
        driver_ids_group = driver_imgs_grouped.get_group('c{}'.format(j))
        paths = os.path.join(base, 'c{}/'.format(j)) + driver_ids_group.img

        if FLAGS.subset:
            paths = paths[:100]
            driver_ids_group = driver_ids_group.iloc[:100]

        driver_ids += driver_ids_group['subject'].tolist()

        for i, path in tqdm(enumerate(paths), total=len(paths)):
            img = load_image(path)
            if i == 0:
                imsave('c{}.jpg'.format(j), img)

            X_train.append(img)
            y_train.append(j)

    X_train = np.array(X_train)
    y_train = np.array(y_train)

    y_train = OneHotEncoder(n_values=NUM_CLASSES) \
        .fit_transform(y_train.reshape(-1, 1)) \
        .toarray()

    return X_train, y_train, driver_ids


In [3]:
def load_test(base):
    X_test = []
    X_test_id = []
    paths = glob.glob('{}*.jpg'.format(base))

    if FLAGS.subset:
        paths = paths[:100]

    print('Reading test images...')
    for i, path in tqdm(enumerate(paths), total=len(paths)):
        id = os.path.basename(path)
        img = load_image(path)

        X_test.append(img)
        X_test_id.append(id)

    X_test = np.array(X_test)
    X_test_id = np.array(X_test_id)

    return X_test, X_test_id


In [8]:
glob.glob?

In [13]:
from time import sleep
from tqdm import tqdm


In [21]:
aa = imresize(imread("~/deeplearning/kaggle/distracted-drivers-tf-master/dataset/imgs/train/c0/img_72999.jpg"), (HEIGHT, WIDTH))

IOError: [Errno 2] No such file or directory: '~/deeplearning/kaggle/distracted-drivers-tf-master/dataset/imgs/train/c0/img_72999.jpg'

In [15]:
for i, path in tqdm(enumerate(paths), total=len(paths)):
    print (i,path)

 35%|███▌      | 873/2489 [00:00<00:00, 8728.07it/s]

0 c0img_44733.jpg
1 c0img_72999.jpg
2 c0img_25094.jpg
3 c0img_69092.jpg
4 c0img_92629.jpg
5 c0img_3370.jpg
6 c0img_67639.jpg
7 c0img_58560.jpg
8 c0img_35779.jpg
9 c0img_10012.jpg
10 c0img_70354.jpg
11 c0img_54617.jpg
12 c0img_26684.jpg
13 c0img_82409.jpg
14 c0img_5585.jpg
15 c0img_19029.jpg
16 c0img_40930.jpg
17 c0img_31547.jpg
18 c0img_87373.jpg
19 c0img_10627.jpg
20 c0img_66355.jpg
21 c0img_97714.jpg
22 c0img_37344.jpg
23 c0img_54848.jpg
24 c0img_59964.jpg
25 c0img_70675.jpg
26 c0img_43317.jpg
27 c0img_13585.jpg
28 c0img_63064.jpg
29 c0img_91058.jpg
30 c0img_79799.jpg
31 c0img_18198.jpg
32 c0img_55108.jpg
33 c0img_87789.jpg
34 c0img_11582.jpg
35 c0img_24152.jpg
36 c0img_28404.jpg
37 c0img_8868.jpg
38 c0img_14949.jpg
39 c0img_41276.jpg
40 c0img_70611.jpg
41 c0img_42895.jpg
42 c0img_87995.jpg
43 c0img_40990.jpg
44 c0img_68866.jpg
45 c0img_71265.jpg
46 c0img_90102.jpg
47 c0img_30954.jpg
48 c0img_3570.jpg
49 c0img_59000.jpg
50 c0img_48323.jpg
51 c0img_13073.jpg
52 c0img_80662.jpg
53 c0im

100%|██████████| 2489/2489 [00:00<00:00, 8643.47it/s]

 c0img_85701.jpg
1560 c0img_13820.jpg
1561 c0img_100922.jpg
1562 c0img_20727.jpg
1563 c0img_375.jpg
1564 c0img_10321.jpg
1565 c0img_74965.jpg
1566 c0img_101702.jpg
1567 c0img_21059.jpg
1568 c0img_25599.jpg
1569 c0img_18847.jpg
1570 c0img_44390.jpg
1571 c0img_30957.jpg
1572 c0img_84894.jpg
1573 c0img_77606.jpg
1574 c0img_89511.jpg
1575 c0img_50586.jpg
1576 c0img_45618.jpg
1577 c0img_14756.jpg
1578 c0img_64182.jpg
1579 c0img_57420.jpg
1580 c0img_20632.jpg
1581 c0img_67972.jpg
1582 c0img_88072.jpg
1583 c0img_29163.jpg
1584 c0img_35182.jpg
1585 c0img_21098.jpg
1586 c0img_67829.jpg
1587 c0img_98853.jpg
1588 c0img_54739.jpg
1589 c0img_981.jpg
1590 c0img_27187.jpg
1591 c0img_90283.jpg
1592 c0img_22984.jpg
1593 c0img_52372.jpg
1594 c0img_26802.jpg
1595 c0img_34032.jpg
1596 c0img_43846.jpg
1597 c0img_50804.jpg
1598 c0img_40071.jpg
1599 c0img_52402.jpg
1600 c0img_95849.jpg
1601 c0img_20847.jpg
1602 c0img_2653.jpg
1603 c0img_29172.jpg
1604 c0img_5015.jpg
1605 c0img_50920.jpg
1606 c0img_71412.jpg


In [ ]:
X_train, y_train, driver_ids = load_train('~/deeplearning/kaggle/distracted-drivers-tf-master/dataset/imgs/train')
X_test, X_test_ids = load_test('imgs/test/')

In [24]:
load_train('imgs/train/')

IOError: File driver_imgs_list.csv does not exist

In [26]:
load_train('~/deeplearning/kaggle/distracted-drivers-tf-master/dataset')

IOError: File driver_imgs_list.csv does not exist